In [1]:
import os 
import sys
import subprocess
import pandas as pd
import numpy as np
import pybedtools
from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from gene2probe import *

/nfs/team205/is10/miniconda/envs/gene2probe/lib/python3.11/site-packages/Bio/Application/__init__.py:40: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [2]:
## specify inputs
gene_ID = 'RPS4Y1'
feature = 'CDS' ## Whether to consider only exons / introns or full gene
gtf = '/nfs/team205/is10/resources/ucsc_genomes/hg38.ncbiRefSeq.gtf' ## Gene annotation in gtf file
## We recommend using RefSeq as this is manually curated and more likely to contain an isoform that is present across most cell types
## Alternatively, one can filter based on RNA-seq data for a cell type/tissue of interest
fasta = '/nfs/team205/is10/resources/ucsc_genomes/hg38.fa' ## Genome in fasta file
snp_db = '/nfs/team205/is10/resources/hg38_dbSNP/hg38_snp151Common_chrY.txt' ## Database of known SNPs
masked_regions_bed = '/nfs/team205/is10/resources/hg38_repeatmasker/hg38_rmsk.bed' ## bed file with coordinates to be excluded
blast_db = '/nfs/team205/is10/projects/thytrans/data/gene2probe/blastdb/hg38_ncbiRefSeq_exons_db' ## Database of all human transcripts to blast against
blast_exec_path = '/nfs/team205/is10/miniconda/envs/gene2probe/bin/'

In [3]:
## additional parameters
probe_length = 50
split_nt = 25 ## Index of nucleotide to split the probe at (start of RHS) - leave None if splitting probe is not needed
min_GC = 0.44
max_GC = 0.72
required_nts = {24: 'T'} ## Dictionary of index (0-based) for required nts - by default, 25th nucleotide must be a T

In [4]:
## Create output directory
out_dir = ('/nfs/team205/is10/projects/thytrans/data/gene2probe/' + gene_ID + '/')
os.makedirs(out_dir, exist_ok=True)

In [5]:
## Read gtf file
gene_anno = read_gtf(gtf)
## Extract regions corresponding to gene of interest (symbol: gene_name, Ensembl ID: gene_ID)
gene_ids = gene_anno['attribute'].apply(extract_feature_from_gtf, feature='gene_name')
roi = gene_anno.iloc[np.where(gene_ids == gene_ID)[0],:]

In [6]:
roi

,seqname,source,feature,start,end,score,strand,frame,attribute
259367,chrY,ncbiRefSeq.2022-10-28,transcript,2841602,2867268,.,+,.,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259368,chrY,ncbiRefSeq.2022-10-28,exon,2841602,2841627,.,+,.,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259369,chrY,ncbiRefSeq.2022-10-28,5UTR,2841602,2841624,.,+,.,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259370,chrY,ncbiRefSeq.2022-10-28,CDS,2841625,2841627,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259371,chrY,ncbiRefSeq.2022-10-28,exon,2842165,2842242,.,+,.,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259372,chrY,ncbiRefSeq.2022-10-28,CDS,2842165,2842242,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259373,chrY,ncbiRefSeq.2022-10-28,exon,2844077,2844257,.,+,.,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259374,chrY,ncbiRefSeq.2022-10-28,CDS,2844077,2844257,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259375,chrY,ncbiRefSeq.2022-10-28,exon,2845646,2845743,.,+,.,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259376,chrY,ncbiRefSeq.2022-10-28,CDS,2845646,2845743,.,+,2,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."


In [7]:
## Subset to feature type of interest
roi = roi[roi['feature']==feature]

In [8]:
roi

,seqname,source,feature,start,end,score,strand,frame,attribute
259370,chrY,ncbiRefSeq.2022-10-28,CDS,2841625,2841627,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259372,chrY,ncbiRefSeq.2022-10-28,CDS,2842165,2842242,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259374,chrY,ncbiRefSeq.2022-10-28,CDS,2844077,2844257,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259376,chrY,ncbiRefSeq.2022-10-28,CDS,2845646,2845743,.,+,2,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259378,chrY,ncbiRefSeq.2022-10-28,CDS,2854600,2854771,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259380,chrY,ncbiRefSeq.2022-10-28,CDS,2865088,2865245,.,+,2,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259382,chrY,ncbiRefSeq.2022-10-28,CDS,2866793,2866891,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""NM_001008.4"";..."
259389,chrY,ncbiRefSeq.2022-10-28,CDS,2841944,2841973,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""XM_047442742...."
259391,chrY,ncbiRefSeq.2022-10-28,CDS,2842165,2842242,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""XM_047442742...."
259393,chrY,ncbiRefSeq.2022-10-28,CDS,2844077,2844257,.,+,0,"gene_id ""RPS4Y1""; transcript_id ""XM_047442742...."


In [9]:
## Convert to bed-style df
roi_bed = gtf_2_bed(roi, name_pref = (gene_ID + '_'))

/nfs/team205/is10/projects/thytrans/scripts/008_visiumHD/gene2probe.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['start'] = df_bed['start'].astype(int) - 1


In [10]:
roi_bed

,seqname,start,end,name,score,strand
0,chrY,2841624,2841627,RPS4Y1_0,.,+
1,chrY,2842164,2842242,RPS4Y1_1,.,+
2,chrY,2844076,2844257,RPS4Y1_2,.,+
3,chrY,2845645,2845743,RPS4Y1_3,.,+
4,chrY,2854599,2854771,RPS4Y1_4,.,+
5,chrY,2865087,2865245,RPS4Y1_5,.,+
6,chrY,2866792,2866891,RPS4Y1_6,.,+
7,chrY,2841943,2841973,RPS4Y1_7,.,+
8,chrY,2842164,2842242,RPS4Y1_8,.,+
9,chrY,2844076,2844257,RPS4Y1_9,.,+


In [11]:
roi_kmers = generate_50mers(roi_bed)

In [12]:
roi_kmers

,seqname,start,end,name,score,strand
0,chrY,2842164,2842214,RPS4Y1_1_0,.,+
1,chrY,2842165,2842215,RPS4Y1_1_1,.,+
2,chrY,2842166,2842216,RPS4Y1_1_2,.,+
3,chrY,2842167,2842217,RPS4Y1_1_3,.,+
4,chrY,2842168,2842218,RPS4Y1_1_4,.,+
...,...,...,...,...,...,...
979,chrY,2866837,2866887,RPS4Y1_13_45,.,+
980,chrY,2866838,2866888,RPS4Y1_13_46,.,+
981,chrY,2866839,2866889,RPS4Y1_13_47,.,+
982,chrY,2866840,2866890,RPS4Y1_13_48,.,+


In [13]:
## Export unfiltered
roi_kmers.to_csv((out_dir + 'kmers_all.csv'))

In [14]:
## Convert to pybedtools
roi_kmers_bed = pybedtools.BedTool.from_dataframe(roi_kmers)

In [15]:
## Remove masked regions
if masked_regions_bed is not None:
    ## Read rmsk
    rmsk = pybedtools.BedTool(masked_regions_bed)
    ## Remove any kmers overlapping any masked regions
    roi_kmers_bed = roi_kmers_bed.intersect(rmsk, v=True)

In [16]:
roi_kmers_filtered = roi_kmers_bed.to_dataframe(names=['seqname', 'start', 'end', 'name', 'score', 'strand'])

In [17]:
roi_kmers_filtered

,seqname,start,end,name,score,strand
0,chrY,2842164,2842214,RPS4Y1_1_0,.,+
1,chrY,2842165,2842215,RPS4Y1_1_1,.,+
2,chrY,2842166,2842216,RPS4Y1_1_2,.,+
3,chrY,2842167,2842217,RPS4Y1_1_3,.,+
4,chrY,2842168,2842218,RPS4Y1_1_4,.,+
...,...,...,...,...,...,...
979,chrY,2866837,2866887,RPS4Y1_13_45,.,+
980,chrY,2866838,2866888,RPS4Y1_13_46,.,+
981,chrY,2866839,2866889,RPS4Y1_13_47,.,+
982,chrY,2866840,2866890,RPS4Y1_13_48,.,+


In [18]:
## Same with SNPs
if snp_db is not None:
    ## Read SNP file (by chromosome as this can be quite large)
    snps = pd.read_csv(snp_db, sep='\t')  
    snps_bed = pybedtools.BedTool.from_dataframe(snps[['#chrom', 'chromStart', 'chromEnd']])
    ## Remove any kmers overlapping any masked regions
    roi_kmers_bed = roi_kmers_bed.intersect(snps_bed, v=True)

In [19]:
roi_kmers_filtered = roi_kmers_bed.to_dataframe(names=['seqname', 'start', 'end', 'name', 'score', 'strand'])

In [20]:
roi_kmers_filtered

,seqname,start,end,name,score,strand
0,chrY,2842164,2842214,RPS4Y1_1_0,.,+
1,chrY,2842165,2842215,RPS4Y1_1_1,.,+
2,chrY,2842166,2842216,RPS4Y1_1_2,.,+
3,chrY,2842167,2842217,RPS4Y1_1_3,.,+
4,chrY,2842168,2842218,RPS4Y1_1_4,.,+
...,...,...,...,...,...,...
979,chrY,2866837,2866887,RPS4Y1_13_45,.,+
980,chrY,2866838,2866888,RPS4Y1_13_46,.,+
981,chrY,2866839,2866889,RPS4Y1_13_47,.,+
982,chrY,2866840,2866890,RPS4Y1_13_48,.,+


In [21]:
## Get DNA for transcript and probe (reverse complement)
## Simultaneously we can monitor GC content and count the longest homopolymer stretch
roi_kmers_seq = roi_kmers_bed.sequence(fi=fasta, s=True) 

kmer_coord = []
transcript_seq = []
probe_seq = []
gc_content = []
gc_content_lhs = []
gc_content_rhs = []
longest_homopol = []
with open(roi_kmers_seq.seqfn) as f:
    for line in f:
        if line.startswith('>'):
            kmer_coord.append(line.strip().replace('>', ''))
        else:
            ## Get sequence
            kmer_seq = Seq(line.strip())
            transcript_seq.append(str(kmer_seq))
            ## Reverse complement - this is the sequence of the probe
            probe_seq.append(str(kmer_seq.reverse_complement()))
            ## Estimate GC content
            gc_content.append(gc_fraction(kmer_seq))
            gc_content_rhs.append(gc_fraction(kmer_seq[0:split_nt])) ## RHS of the probe is LHS of the target - GC content is the same
            gc_content_lhs.append(gc_fraction(kmer_seq[split_nt: probe_length])) ## RHS of the probe is LHS of the target - GC content is the same
            ## Estimate longest homopolymer
            longest_homopol.append(get_longest_homopolymer(kmer_seq))

## Incorporate sequences and stats into the dataframe
roi_kmers_filtered['kmer_coord'] = kmer_coord
roi_kmers_filtered['transcript_seq'] = transcript_seq
roi_kmers_filtered['probe_seq'] = probe_seq
roi_kmers_filtered['GC_content_full'] = gc_content
roi_kmers_filtered['GC_content_LHS'] = gc_content_lhs
roi_kmers_filtered['GC_content_RHS'] = gc_content_rhs
roi_kmers_filtered['longest_homopolymer'] = longest_homopol

In [22]:
roi_kmers_filtered

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,GC_content_LHS,GC_content_RHS,longest_homopolymer
0,chrY,2842164,2842214,RPS4Y1_1_0,.,+,chrY:2842164-2842214(+),GCCCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGC...,CAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCCCC...,0.62,0.60,0.64,4
1,chrY,2842165,2842215,RPS4Y1_1_1,.,+,chrY:2842165-2842215(+),CCCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCA...,CCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCCC...,0.62,0.64,0.60,4
2,chrY,2842166,2842216,RPS4Y1_1_2,.,+,chrY:2842166-2842216(+),CCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCAT...,TCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCC...,0.60,0.60,0.60,4
3,chrY,2842167,2842217,RPS4Y1_1_3,.,+,chrY:2842167-2842217(+),CGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCATT...,ATCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGC...,0.58,0.56,0.60,4
4,chrY,2842168,2842218,RPS4Y1_1_4,.,+,chrY:2842168-2842218(+),GGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCATTG...,CATCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGG...,0.58,0.56,0.60,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,chrY,2866837,2866887,RPS4Y1_13_45,.,+,chrY:2866837-2866887(+),CTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAACAGA...,CTGCTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAG...,0.50,0.60,0.40,3
980,chrY,2866838,2866888,RPS4Y1_13_46,.,+,chrY:2866838-2866888(+),TTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAACAGAG...,ACTGCTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACA...,0.48,0.60,0.36,3
981,chrY,2866839,2866889,RPS4Y1_13_47,.,+,chrY:2866839-2866889(+),TACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAACAGAGC...,CACTGCTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAAC...,0.50,0.60,0.40,3
982,chrY,2866840,2866890,RPS4Y1_13_48,.,+,chrY:2866840-2866890(+),ACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAACAGAGCA...,CCACTGCTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAA...,0.52,0.64,0.40,3


In [23]:
## Mark probes matching nucleotide requirement:
has_required_nt = [True for i in range(roi_kmers_filtered.shape[0])]
for nt_idx in required_nts.keys():
    for i in range(roi_kmers_filtered.shape[0]):
        if roi_kmers_filtered['probe_seq'][i][nt_idx] != required_nts[nt_idx]:
            has_required_nt[i] = False

In [24]:
roi_kmers_filtered['has_required_nts'] = has_required_nt

In [25]:
roi_kmers_filtered['has_required_nts'].value_counts()

has_required_nts
False    728
True     256
Name: count, dtype: int64

In [26]:
## Export kmers before filtering
roi_kmers_filtered.to_csv((out_dir + 'kmers_candidates_unfiltered.csv'))

In [27]:
roi_kmers_filtered

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,GC_content_LHS,GC_content_RHS,longest_homopolymer,has_required_nts
0,chrY,2842164,2842214,RPS4Y1_1_0,.,+,chrY:2842164-2842214(+),GCCCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGC...,CAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCCCC...,0.62,0.60,0.64,4,True
1,chrY,2842165,2842215,RPS4Y1_1_1,.,+,chrY:2842165-2842215(+),CCCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCA...,CCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCCC...,0.62,0.64,0.60,4,False
2,chrY,2842166,2842216,RPS4Y1_1_2,.,+,chrY:2842166-2842216(+),CCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCAT...,TCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCC...,0.60,0.60,0.60,4,False
3,chrY,2842167,2842217,RPS4Y1_1_3,.,+,chrY:2842167-2842217(+),CGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCATT...,ATCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGC...,0.58,0.56,0.60,4,False
4,chrY,2842168,2842218,RPS4Y1_1_4,.,+,chrY:2842168-2842218(+),GGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCATTG...,CATCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGG...,0.58,0.56,0.60,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,chrY,2866837,2866887,RPS4Y1_13_45,.,+,chrY:2866837-2866887(+),CTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAACAGA...,CTGCTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAG...,0.50,0.60,0.40,3,True
980,chrY,2866838,2866888,RPS4Y1_13_46,.,+,chrY:2866838-2866888(+),TTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAACAGAG...,ACTGCTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACA...,0.48,0.60,0.36,3,False
981,chrY,2866839,2866889,RPS4Y1_13_47,.,+,chrY:2866839-2866889(+),TACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAACAGAGC...,CACTGCTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAAC...,0.50,0.60,0.40,3,True
982,chrY,2866840,2866890,RPS4Y1_13_48,.,+,chrY:2866840-2866890(+),ACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAACAGAGCA...,CCACTGCTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAA...,0.52,0.64,0.40,3,False


In [28]:
## Filter for GC content
roi_kmers_filtered = roi_kmers_filtered[(roi_kmers_filtered['GC_content_full'] > min_GC) & 
(roi_kmers_filtered['GC_content_LHS'] > min_GC) & (roi_kmers_filtered['GC_content_RHS'] > min_GC) & 
(roi_kmers_filtered['GC_content_full'] < max_GC) & 
(roi_kmers_filtered['GC_content_LHS'] < max_GC) & (roi_kmers_filtered['GC_content_RHS'] < max_GC)].reset_index(drop=True)

In [29]:
roi_kmers_filtered

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,GC_content_LHS,GC_content_RHS,longest_homopolymer,has_required_nts
0,chrY,2842164,2842214,RPS4Y1_1_0,.,+,chrY:2842164-2842214(+),GCCCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGC...,CAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCCCC...,0.62,0.60,0.64,4,True
1,chrY,2842165,2842215,RPS4Y1_1_1,.,+,chrY:2842165-2842215(+),CCCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCA...,CCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCCC...,0.62,0.64,0.60,4,False
2,chrY,2842166,2842216,RPS4Y1_1_2,.,+,chrY:2842166-2842216(+),CCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCAT...,TCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCC...,0.60,0.60,0.60,4,False
3,chrY,2842167,2842217,RPS4Y1_1_3,.,+,chrY:2842167-2842217(+),CGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCATT...,ATCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGC...,0.58,0.56,0.60,4,False
4,chrY,2842168,2842218,RPS4Y1_1_4,.,+,chrY:2842168-2842218(+),GGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCATTG...,CATCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGG...,0.58,0.56,0.60,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,chrY,2866825,2866875,RPS4Y1_13_33,.,+,chrY:2866825-2866875(+),AAGGGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGG...,GTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAATGC...,0.50,0.52,0.48,3,True
402,chrY,2866826,2866876,RPS4Y1_13_34,.,+,chrY:2866826-2866876(+),AGGGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGC...,GGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAATG...,0.52,0.56,0.48,3,True
403,chrY,2866827,2866877,RPS4Y1_13_35,.,+,chrY:2866827-2866877(+),GGGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCC...,TGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAAT...,0.52,0.56,0.48,3,False
404,chrY,2866828,2866878,RPS4Y1_13_36,.,+,chrY:2866828-2866878(+),GGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCA...,TTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAA...,0.50,0.52,0.48,2,True


In [30]:
## Filter for required nucleotides
roi_kmers_filtered = roi_kmers_filtered[roi_kmers_filtered['has_required_nts']==True].reset_index(drop=True)

In [31]:
## Candidate kmers
roi_kmers_filtered

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,GC_content_LHS,GC_content_RHS,longest_homopolymer,has_required_nts
0,chrY,2842164,2842214,RPS4Y1_1_0,.,+,chrY:2842164-2842214(+),GCCCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGC...,CAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCCCC...,0.62,0.60,0.64,4,True
1,chrY,2842174,2842224,RPS4Y1_1_10,.,+,chrY:2842174-2842224(+),CCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGCATTGGATGCT...,GTCAAGCATCCAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTC...,0.52,0.56,0.48,3,True
2,chrY,2844076,2844126,RPS4Y1_2_0,.,+,chrY:2844076-2844126(+),GCACCTCGTCCATCGACAGGTCCCCACAAGCTGAGGGAATGTCTTC...,AGAGGAAGACATTCCCTCAGCTTGTGGGGACCTGTCGATGGACGAG...,0.58,0.48,0.68,4,True
3,chrY,2844078,2844128,RPS4Y1_2_2,.,+,chrY:2844078-2844128(+),ACCTCGTCCATCGACAGGTCCCCACAAGCTGAGGGAATGTCTTCCT...,TCAGAGGAAGACATTCCCTCAGCTTGTGGGGACCTGTCGATGGACG...,0.56,0.48,0.64,4,True
4,chrY,2844079,2844129,RPS4Y1_2_3,.,+,chrY:2844079-2844129(+),CCTCGTCCATCGACAGGTCCCCACAAGCTGAGGGAATGTCTTCCTC...,ATCAGAGGAAGACATTCCCTCAGCTTGTGGGGACCTGTCGATGGAC...,0.56,0.48,0.64,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,chrY,2866801,2866851,RPS4Y1_13_9,.,+,chrY:2866801-2866851(+),CCTTGGATTTCCCTGCCCAGGGGAAAGGGCATTCGACTTACTGTTG...,TCAGCAACAGTAAGTCGAATGCCCTTTCCCCTGGGCAGGGAAATCC...,0.54,0.48,0.60,4,True
92,chrY,2866825,2866875,RPS4Y1_13_33,.,+,chrY:2866825-2866875(+),AAGGGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGG...,GTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAATGC...,0.50,0.52,0.48,3,True
93,chrY,2866826,2866876,RPS4Y1_13_34,.,+,chrY:2866826-2866876(+),AGGGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGC...,GGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAATG...,0.52,0.56,0.48,3,True
94,chrY,2866828,2866878,RPS4Y1_13_36,.,+,chrY:2866828-2866878(+),GGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCA...,TTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAA...,0.50,0.52,0.48,2,True


In [32]:
roi_kmers_filtered.to_csv((out_dir + 'kmers_candidates_filtered.csv'))

In [33]:
## We are exporting the transcript sequence as that's the one that has to be blasted against the human transcriptome
write_fasta(roi_kmers_filtered['name'], roi_kmers_filtered['transcript_seq'], (out_dir + 'kmers_candidates_filtered_transcript_seqs.fa'))

In [34]:
command = [
    (blast_exec_path + 'blastn'),
    '-query', (out_dir + 'kmers_candidates_filtered_transcript_seqs.fa'), ## Sequences of targeted regions
    '-db', blast_db,                     # BLAST database
    '-out', (out_dir + 'kmers_candidates_filtered_blast_output.txt'),  # Output file
    '-outfmt', '6',
    '-strand', 'plus',
    '-evalue', '1e-3',  # More lenient e-value to detect potential off-targets
    '-dust', 'no'       # Turn off low-complexity filter
]

# Run the command
result = subprocess.run(command, capture_output=True, text=True)

In [35]:
## Read in the blast output
blast_columns = [
    "name",   # Query Seq-id
    "sseqid",   # Subject Seq-id
    "pident",   # Percentage of identical matches
    "length",   # Alignment length
    "mismatch", # Number of mismatches
    "gapopen",  # Number of gap openings
    "qstart",   # Start of alignment in query
    "qend",     # End of alignment in query
    "sstart",   # Start of alignment in subject
    "send",     # End of alignment in subject
    "evalue",   # Expect value
    "bitscore"  # Bit score
]

blast_res = pd.read_csv(
    (out_dir + 'kmers_candidates_filtered_blast_output.txt'),
    sep='\t', 
    header=None, 
    names=blast_columns
    )

In [36]:
## Extract the gene ID
blast_res['sgeneid'] = blast_res['sseqid'].str.split('::').str[0]

In [37]:
blast_res['sgeneid'].value_counts()

sgeneid
RPS4Y1      192
RPS4Y2       22
RPS4Y1P1     14
RPS4XP8      10
Name: count, dtype: int64

In [38]:
blast_res['evalue'].describe()

count    2.380000e+02
mean     2.001727e-08
std      2.026337e-07
min      1.300000e-18
25%      1.300000e-18
50%      1.300000e-18
75%      1.300000e-18
max      2.210000e-06
Name: evalue, dtype: float64

In [39]:
offtargets = blast_res['name'][blast_res['sgeneid']!=gene_ID].unique()

In [40]:
offtargets

array(['RPS4Y1_1_10', 'RPS4Y1_2_0', 'RPS4Y1_2_2', 'RPS4Y1_2_3',
       'RPS4Y1_2_8', 'RPS4Y1_2_13', 'RPS4Y1_2_44', 'RPS4Y1_2_50',
       'RPS4Y1_4_20', 'RPS4Y1_4_27', 'RPS4Y1_4_35', 'RPS4Y1_4_39',
       'RPS4Y1_4_42', 'RPS4Y1_4_49', 'RPS4Y1_4_50', 'RPS4Y1_6_2',
       'RPS4Y1_6_8', 'RPS4Y1_6_9', 'RPS4Y1_8_10', 'RPS4Y1_9_0',
       'RPS4Y1_9_2', 'RPS4Y1_9_3', 'RPS4Y1_9_8', 'RPS4Y1_9_13',
       'RPS4Y1_9_44', 'RPS4Y1_9_50', 'RPS4Y1_11_20', 'RPS4Y1_11_27',
       'RPS4Y1_11_35', 'RPS4Y1_11_39', 'RPS4Y1_11_42', 'RPS4Y1_11_49',
       'RPS4Y1_11_50', 'RPS4Y1_13_2', 'RPS4Y1_13_8', 'RPS4Y1_13_9'],
      dtype=object)

In [41]:
## Remove off targets
roi_kmers_filtered = roi_kmers_filtered[roi_kmers_filtered['name'].isin(offtargets)==False].reset_index(drop=True)

In [42]:
roi_kmers_filtered 

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,GC_content_LHS,GC_content_RHS,longest_homopolymer,has_required_nts
0,chrY,2842164,2842214,RPS4Y1_1_0,.,+,chrY:2842164-2842214(+),GCCCGGGGCCCCAAGAAGCACTTAAAGCGTGTTGCAGCGCCGAAGC...,CAATGCTTCGGCGCTGCAACACGCTTTAAGTGCTTCTTGGGGCCCC...,0.62,0.60,0.64,4,True
1,chrY,2845647,2845697,RPS4Y1_3_2,.,+,chrY:2845647-2845697(+),GTCATCAGCATCGAGAAGACAGGTGAACATTTCCGCCTGGTCTATG...,GTGTCATAGACCAGGCGGAAATGTTCACCTGTCTTCTCGATGCTGA...,0.50,0.48,0.52,3,True
2,chrY,2845648,2845698,RPS4Y1_3_3,.,+,chrY:2845648-2845698(+),TCATCAGCATCGAGAAGACAGGTGAACATTTCCGCCTGGTCTATGA...,GGTGTCATAGACCAGGCGGAAATGTTCACCTGTCTTCTCGATGCTG...,0.50,0.52,0.48,3,True
3,chrY,2845650,2845700,RPS4Y1_3_5,.,+,chrY:2845650-2845700(+),ATCAGCATCGAGAAGACAGGTGAACATTTCCGCCTGGTCTATGACA...,TTGGTGTCATAGACCAGGCGGAAATGTTCACCTGTCTTCTCGATGC...,0.48,0.48,0.48,3,True
4,chrY,2845665,2845715,RPS4Y1_3_20,.,+,chrY:2845665-2845715(+),ACAGGTGAACATTTCCGCCTGGTCTATGACACCAAGGGCCGTTTTG...,ACAGCAAAACGGCCCTTGGTGTCATAGACCAGGCGGAAATGTTCAC...,0.52,0.52,0.52,4,True
5,chrY,2845668,2845718,RPS4Y1_3_23,.,+,chrY:2845668-2845718(+),GGTGAACATTTCCGCCTGGTCTATGACACCAAGGGCCGTTTTGCTG...,TGAACAGCAAAACGGCCCTTGGTGTCATAGACCAGGCGGAAATGTT...,0.52,0.52,0.52,4,True
6,chrY,2845670,2845720,RPS4Y1_3_25,.,+,chrY:2845670-2845720(+),TGAACATTTCCGCCTGGTCTATGACACCAAGGGCCGTTTTGCTGTT...,GGTGAACAGCAAAACGGCCCTTGGTGTCATAGACCAGGCGGAAATG...,0.52,0.56,0.48,4,True
7,chrY,2845673,2845723,RPS4Y1_3_28,.,+,chrY:2845673-2845723(+),ACATTTCCGCCTGGTCTATGACACCAAGGGCCGTTTTGCTGTTCAC...,TGCGGTGAACAGCAAAACGGCCCTTGGTGTCATAGACCAGGCGGAA...,0.54,0.56,0.52,4,True
8,chrY,2845674,2845724,RPS4Y1_3_29,.,+,chrY:2845674-2845724(+),CATTTCCGCCTGGTCTATGACACCAAGGGCCGTTTTGCTGTTCACC...,ATGCGGTGAACAGCAAAACGGCCCTTGGTGTCATAGACCAGGCGGA...,0.54,0.56,0.52,4,True
9,chrY,2845692,2845742,RPS4Y1_3_47,.,+,chrY:2845692-2845742(+),GACACCAAGGGCCGTTTTGCTGTTCACCGCATCACAGTGGAAGAGG...,TTTGCCTCTTCCACTGTGATGCGGTGAACAGCAAAACGGCCCTTGG...,0.54,0.52,0.56,4,True


In [43]:
## Sort in increasing homopolymer length
roi_kmers_filtered = roi_kmers_filtered.sort_values('longest_homopolymer', ascending=True).reset_index(drop=True)

In [44]:
roi_kmers_filtered

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,GC_content_LHS,GC_content_RHS,longest_homopolymer,has_required_nts
0,chrY,2866828,2866878,RPS4Y1_6_36,.,+,chrY:2866828-2866878(+),GGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCA...,TTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAA...,0.50,0.52,0.48,2,True
1,chrY,2866828,2866878,RPS4Y1_13_36,.,+,chrY:2866828-2866878(+),GGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCA...,TTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAA...,0.50,0.52,0.48,2,True
2,chrY,2865092,2865142,RPS4Y1_5_5,.,+,chrY:2865092-2865142(+),TTGTGTATGGTGATTGGTGGAGCCAACCTCGGTCGTGTTGGTGTGA...,GTGATCACACCAACACGACCGAGGTTGGCTCCACCAATCACCATAC...,0.52,0.56,0.48,2,True
3,chrY,2865092,2865142,RPS4Y1_12_5,.,+,chrY:2865092-2865142(+),TTGTGTATGGTGATTGGTGGAGCCAACCTCGGTCGTGTTGGTGTGA...,GTGATCACACCAACACGACCGAGGTTGGCTCCACCAATCACCATAC...,0.52,0.56,0.48,2,True
4,chrY,2866834,2866884,RPS4Y1_6_42,.,+,chrY:2866834-2866884(+),CGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAAAC...,CTCTGTTTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAA...,0.50,0.52,0.48,3,True
5,chrY,2845647,2845697,RPS4Y1_10_2,.,+,chrY:2845647-2845697(+),GTCATCAGCATCGAGAAGACAGGTGAACATTTCCGCCTGGTCTATG...,GTGTCATAGACCAGGCGGAAATGTTCACCTGTCTTCTCGATGCTGA...,0.50,0.48,0.52,3,True
6,chrY,2845648,2845698,RPS4Y1_10_3,.,+,chrY:2845648-2845698(+),TCATCAGCATCGAGAAGACAGGTGAACATTTCCGCCTGGTCTATGA...,GGTGTCATAGACCAGGCGGAAATGTTCACCTGTCTTCTCGATGCTG...,0.50,0.52,0.48,3,True
7,chrY,2845650,2845700,RPS4Y1_10_5,.,+,chrY:2845650-2845700(+),ATCAGCATCGAGAAGACAGGTGAACATTTCCGCCTGGTCTATGACA...,TTGGTGTCATAGACCAGGCGGAAATGTTCACCTGTCTTCTCGATGC...,0.48,0.48,0.48,3,True
8,chrY,2854609,2854659,RPS4Y1_11_10,.,+,chrY:2854609-2854659(+),GCAAAGTGAGGAAGATTACTGTGGGAGTGAAGGGAATCCCTCACCT...,CACCAGGTGAGGGATTCCCTTCACTCCCACAGTAATCTTCCTCACT...,0.52,0.56,0.48,3,True
9,chrY,2854613,2854663,RPS4Y1_11_14,.,+,chrY:2854613-2854663(+),AGTGAGGAAGATTACTGTGGGAGTGAAGGGAATCCCTCACCTGGTG...,GAGTCACCAGGTGAGGGATTCCCTTCACTCCCACAGTAATCTTCCT...,0.52,0.56,0.48,3,True


In [45]:
selected_probes_list = []
df = roi_kmers_filtered.copy()
i = 0
while i < 3:
    selected_probes_list.append(df.iloc[0:1,:])
    df = remove_overlapping_probes(df,
                                   df['name'][0], 
                                   offset=100).reset_index(drop=True).copy()
    i +=1

/nfs/team205/is10/projects/thytrans/scripts/008_visiumHD/gene2probe.py:117: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  start = int(probe_df['start'].iloc[i])
/nfs/team205/is10/projects/thytrans/scripts/008_visiumHD/gene2probe.py:118: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  end = int(probe_df['end'].iloc[i])


In [46]:
selected_probes_df = pd.concat(selected_probes_list, axis=0).reset_index(drop=True)

In [47]:
selected_probes_df

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,GC_content_LHS,GC_content_RHS,longest_homopolymer,has_required_nts
0,chrY,2866828,2866878,RPS4Y1_6_36,.,+,chrY:2866828-2866878(+),GGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCA...,TTGGTGGCCAGCCTCTTATCTCTCTCTTCAGCAACAGTAAGTCGAA...,0.50,0.52,0.48,2,True
1,chrY,2865092,2865142,RPS4Y1_5_5,.,+,chrY:2865092-2865142(+),TTGTGTATGGTGATTGGTGGAGCCAACCTCGGTCGTGTTGGTGTGA...,GTGATCACACCAACACGACCGAGGTTGGCTCCACCAATCACCATAC...,0.52,0.56,0.48,2,True
2,chrY,2845647,2845697,RPS4Y1_10_2,.,+,chrY:2845647-2845697(+),GTCATCAGCATCGAGAAGACAGGTGAACATTTCCGCCTGGTCTATG...,GTGTCATAGACCAGGCGGAAATGTTCACCTGTCTTCTCGATGCTGA...,0.50,0.48,0.52,3,True


In [48]:
for seq in selected_probes_df['transcript_seq']:
    print(seq)

GGCATTCGACTTACTGTTGCTGAAGAGAGAGATAAGAGGCTGGCCACCAA
TTGTGTATGGTGATTGGTGGAGCCAACCTCGGTCGTGTTGGTGTGATCAC
GTCATCAGCATCGAGAAGACAGGTGAACATTTCCGCCTGGTCTATGACAC


In [52]:
## Make split probes
selected_probes_df['probe_seq_LHS'] = [seq[0:split_nt] for seq in selected_probes_df['probe_seq']]
selected_probes_df['probe_seq_RHS'] = [seq[split_nt: probe_length] for seq in selected_probes_df['probe_seq']]

In [53]:
## Also add gene_ID for completeness
selected_probes_df['gene_ID'] = gene_ID

In [54]:
## Export selected probes as dataframe:
selected_probes_df.to_csv((out_dir + 'kmers_selected_probes.csv'))